# Imports

In [ ]:
from pathlib import Path
import pyvista as pv
import pandas as pd

from geograypher.constants import DATA_FOLDER
from geograypher.cameras.derived_cameras import COLMAPCameraSet
from geograypher.predictors.derived_segmentors import TabularRectangleSegmentor

# Input data

In [ ]:
COLMAP_EXAMPLE_DATA_FOLDER = Path(DATA_FOLDER, "COLMAP_bird_detection")
# These are saved from the pycolmap model
CAMERAS_FILE = Path(COLMAP_EXAMPLE_DATA_FOLDER, "cameras.txt")
IMAGES_FILE = Path(COLMAP_EXAMPLE_DATA_FOLDER, "images.txt")
POINTS_FILE = Path(COLMAP_EXAMPLE_DATA_FOLDER, "points3D.txt")
# This is saved from the df of predictions
DETECTIONS_FILE = Path(COLMAP_EXAMPLE_DATA_FOLDER, "preds.csv")
IMAGE_FOLDER = Path(COLMAP_EXAMPLE_DATA_FOLDER, "images")

# Show the COLMAP point cloud and the camera set

In [ ]:
# Create a set of cameras from the COLMAP exports
camera_set = COLMAPCameraSet(
    cameras_file=CAMERAS_FILE,
    images_file=IMAGES_FILE,
    image_folder=IMAGE_FOLDER,
    validate_images=False,
)

# Load in the COLMAP sparse keypoints
points_df = pd.read_csv(
    POINTS_FILE,
    sep=" ",
    header=None,
    skiprows=[0, 1, 2],
    names=("POINTS3D_ID", "X", "Y", "Z", "R", "G", "B"),
    usecols=list(range(7)),
)
# Create a pyvista object from the keypoints
points_xyz = points_df[["X", "Y", "Z"]].to_numpy()
points_rgb = points_df[["R", "G", "B"]].to_numpy()
points_pv = pv.PointSet(points_xyz)

# Create a plotter to show the 3D data
plotter = pv.Plotter()
pv.start_xvfb()
# Show the keypoints
plotter.add_mesh(points_pv, scalars=points_rgb, rgb=True)
# Show the cameras
camera_set.vis(show=True, frustum_scale=10, plotter=plotter)

# Show the rays from each detection and their triangulated locations

In [ ]:
# This object reads the detections exported in the DeepForest format
detector = TabularRectangleSegmentor(
    detection_file_or_folder=DETECTIONS_FILE, image_folder=IMAGE_FOLDER, image_shape=(8256, 5504), split_bbox=False
)

# Create a new plotter
plotter = pv.Plotter()
# Show the mesh
plotter.add_mesh(points_pv, scalars=points_rgb, rgb=True)
# Identify correspondences between detections and show them
camera_set.triangulate_detections(
    detector=detector, plotter=plotter, similarity_threshold_meters=0.1, vis_ray_length_meters=80
)
# Show the cameras
# TODO determine why this must be shown after the triangulation stage for proper visualization
camera_set.vis(show=True, frustum_scale=10, plotter=plotter)